In [1]:
import numpy as np
import math
from decimal import Context, setcontext, getcontext, Decimal, ROUND_HALF_EVEN

In [2]:
myothercontext = Context(prec=7)
setcontext(myothercontext)

In [3]:
#Setup 
Neutral = Decimal(1)
Annual_fix = Decimal(365/360)
Nper_in_a_year = Decimal(1/12) #Not valid in Peru and peruvians
Periods = Decimal(12) #Not valid in Peru and peruvians
Significant_numbers = 4

In [4]:
#to test
interest = Decimal(0.0125)
nper = 180
grace = Decimal(6)
insurance_monthly_fix = Decimal(3)
property_value = Decimal(5000)
loan = Decimal(4000)
unsolved_factor = Decimal(1)
credit_life_insurance_factor = Decimal(0.000068)
credit_fire_insurance_factor = Decimal(0.00168)

In [5]:
credit_property_insurance_factor = {
    "dpto": Decimal(0.9),
    "casa": Decimal(0.75),
}

In [6]:
#aux functions only
def thirdpow(a,b,c):
    return pow(pow(a,b),c) #Excel does a ^ b ^ c as [(a ^ b) ^ c] instead of [a ^ (b ^ c)]

def get_monthly_rate(annual_rate):
    getcontext().rounding = ROUND_HALF_EVEN
    return thirdpow(Neutral + annual_rate, Annual_fix, Nper_in_a_year ) - Neutral

def get_monthly_rate_amount_applied(annual_rate, grace):
    return round(pow((Neutral + Decimal(annual_rate)), Decimal(unsolved_factor_k(grace)/Periods)) - Neutral, Significant_numbers)



class SimulatedPayment:
    def __init__(self, property_value, loan, nper, interest, grace, property_type):
        self.interest = interest
        self.grace = grace
        self.nper = nper
        self.loan = loan
        self.property_value = property_value
        self.property_type = property_type
        self.payment = self.get_payment()
                 
    def unsolved_factor_k(self, grace):
        unsolved_factor = Neutral
        for x in range(int(grace)):
            unsolved_factor += unsolved_factor * get_monthly_rate(interest)
        return unsolved_factor
    
    def get_payment(self):
        getcontext().rounding = ROUND_HALF_EVEN
        monthly_rate_aux = get_monthly_rate(self.interest)
        num = self.unsolved_factor_k(int(self.grace)) * monthly_rate_aux
        dem = Neutral - (pow((Neutral + monthly_rate_aux), self.nper * -Neutral))
        return num / dem * loan
    
    credit_property_insurance_factor = {
        "dpto": Decimal(0.9),
        "casa": Decimal(0.75),
    }
    
    def get_fire_insurance(self, per):
        fire_rate = credit_fire_insurance_factor * self.property_value * \
                    Nper_in_a_year * self.credit_property_insurance_factor[self.property_type]

        first_period_value = fire_rate * (grace + insurance_monthly_fix)
        return first_period_value if per == 0 else fire_rate
    
    """def get_factors(self):
        rate = get_monthly_rate(self.interest)
        unsolved_factor_base = self.unsolved_factor_k(self.grace)
        factors_base = []
        for i in range(self.nper):
            unsolved_factor_base -= unsolved_factor_base * rate
            factors_base.append(unsolved_factor_base)
        return factors_base
    """
            
    
    def get_life_insurance(self, per):
        life_rate = self.unsolved_factor_k(self.grace) * self.loan * \
                             credit_life_insurance_factor 
        
        first_period_value = life_rate * (self.grace + insurance_monthly_fix)
        return first_period_value if per == 0 else life_rate     
    
    def get_total(self, term):
        return self.get_payment() + self.get_fire_insurance(term) + self.get_life_insurance(term)
    
    def build_payments(self):
        factors = []
        for k in range(self.nper):
            factors.append(float(self.get_total(k)))
        return [(float(self.loan * -1))] + [0] * int(self.grace) + factors
    
    def get_cae(self):
        return np.irr(self.build_payments()) * int(Periods) * 100
    
    def get_ctc(self):
        return sum(self.build_payments()) + float(loan)
        

    def __repr__(self):
        return f'SimulatedPayment{self.__dict__}'
    
credit = SimulatedPayment(property_value, loan, nper, interest, grace, "dpto")

print(credit.get_cae())
print(credit.get_ctc())
print(credit.get_factors())


1.749462803393076
4589.633740000007
[Decimal('1.005259'), Decimal('1.004203'), Decimal('1.003149'), Decimal('1.002096'), Decimal('1.001044'), Decimal('0.9999929'), Decimal('0.9989429'), Decimal('0.9978940'), Decimal('0.9968462'), Decimal('0.9957995'), Decimal('0.9947539'), Decimal('0.9937094'), Decimal('0.9926660'), Decimal('0.9916237'), Decimal('0.9905825'), Decimal('0.9895424'), Decimal('0.9885034'), Decimal('0.9874655'), Decimal('0.9864287'), Decimal('0.9853930'), Decimal('0.9843583'), Decimal('0.9833247'), Decimal('0.9822922'), Decimal('0.9812608'), Decimal('0.9802305'), Decimal('0.9792013'), Decimal('0.9781731'), Decimal('0.9771460'), Decimal('0.9761200'), Decimal('0.9750951'), Decimal('0.9740712'), Decimal('0.9730484'), Decimal('0.9720267'), Decimal('0.9710061'), Decimal('0.9699865'), Decimal('0.9689680'), Decimal('0.9679506'), Decimal('0.9669343'), Decimal('0.9659190'), Decimal('0.9649048'), Decimal('0.9638916'), Decimal('0.9628795'), Decimal('0.9618685'), Decimal('0.9608585'), 